In [1]:
import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import os
import re
import plotly.tools as tls
import plotly.figure_factory as ff
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
import nltk
nltk.download('stopwords')
from string import punctuation
from collections import OrderedDict
import seaborn as sns
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import json
import random

init_notebook_mode(connected=True) #do not miss this line
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yunzehui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
//anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [59]:
start_month='2017-01'
end_month='2020-02'
    
def datelist(start_month, end_month):
    start_year = int(start_month[:4])
    start_month = int(start_month[-2:])
    end_year = int(end_month[:4])
    end_month = int(end_month[-2:])
    if start_year == end_year:
        month_range = range(start_month, end_month + 1)
        date_list = ["{year}-{month:0=2d}".format(year=str(start_year), month=M) for M in month_range]
        return date_list
    year_range = range(start_year + 1, end_year)
    start_year_month_range = range(start_month, 13)
    end_year_month_range = range(1, end_month + 1)
    date_list = ["{year}-{month:0=2d}".format(year=str(start_year), month=M) for M in start_year_month_range]
    date_list += ["{year}-{month:0=2d}".format(year=str(Y), month=M) for Y in year_range for M in range(1, 13)]
    date_list += ["{year}-{month:0=2d}".format(year=str(end_year), month=M) for M in end_year_month_range]
    return date_list

dl = datelist(start_month, end_month)

filepath=[]
for i in range(len(dl)):
    pathi = 'result/'+dl[i]+'.csv'
    filepath.append(pathi)

In [60]:
tweets = pd.DataFrame(columns=['text', 'label', 'score', 'elapsed_time'])
for i in range(len(dl)):
    read = pd.read_csv(filepath[i],index_col=0)
    tweets = tweets.append(read)

In [62]:
score_dist = tweets["label"].value_counts().to_frame()
score_dist.columns = ["count"]
score_dist = score_dist.reset_index().sort_values(by = "index" ,ascending = False)

layout = go.Layout(dict(title = "Scores distribution",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     gridwidth = 2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     gridwidth = 2),
                        )
                  )

trace = go.Bar(x = score_dist["index"],
               y = score_dist["count"],
               marker = dict(line = dict(width = 1,color = "black"),
                             color = "red")
              )


fig = go.Figure(data = [trace],layout = layout)
py.offline.iplot(fig)

In [66]:
random.sample(list(tweets[tweets["label"]=='POSITIVE']['text']),10)

['team enjoyed participating better half brunch week ago thank partnership',
 'really enjoyable afternoon presenting expert brand social mediaexpecting twitter follower tonight',
 'here latest investment newsletter colleague prospect',
 'justice department asked well fargo probe unit wsj',
 'aixigo roboinvestor london next week come meet u',
 'new post benefit crowe crm microsoft dynamic',
 'radiant logistics rlgt holding maintained kestrel investment management corp modera trimmed stake amgen amgn million stock declined',
 'concentrated extended lifespan among super rich prompting prioritize service simonefoxman',
 'colored diamond star recent christie auction paragon international note',
 'midweek hi']

In [67]:
random.sample(list(tweets[tweets["label"]=='NEUTRAL']['text']),10)

['could done differently find',
 'hdfc bank ranked euromoney private banking survey',
 'volatility',
 'stock market volatility continues sp finished lowest price level since february read latest paper affiliate u bank',
 'iam platform curated tweet itr filing website efile tax return august deadline moneycontrol',
 'fft invests weight watcher international wtw',
 'match day hollandladies suffolkwomens league pm g pitch needhammktfc bloomfield ip da match day info sponsored dean j last ltd',
 'day alexa google assistant able manage wealth voice command place tool brandnew world read vasyl soloschucks new article finplan',
 'secure money advisor premier retirement planning partner western pennsylvania pittsburgh area pittsburgh schedule meeting u view',
 'perpetualltd dump ampau amid broader malaise writes tonyboydafr']

In [68]:
random.sample(list(tweets[tweets["label"]=='NEGATIVE']['text']),10)

['struggling pay credit card personal loan isnt way hack use save money find',
 'financial advisor spokane wa get financial planning job',
 'problem government capitalist slaver people hate',
 'spent bit time last year firm direct line national stock exchange got see process firsthand bought sold share talking lates technology csv file',
 'uk investor longer sitting sideline rising geopolitical tension put risk therefore gold back spotlight also back top agenda',
 'millionaire myth broken',
 'proud sponsor leicestercurry award excited announce mattioli wood consultant nimmy uppal amal mashru guest bbcleicester live tonight pm dont miss',
 'really good see bank moving away conflict vertically integrated service selling insurance',
 'want feel better spend money saving time wsj',
 'internet ecommerce behemoth racing leverage massive customer base grab bigger share financial service sector burgeoning market baidu struggled catch bigger rival caixin']